In [35]:
import osmnx as ox
import matplotlib.pyplot as plt
import geopandas as gpd
import csv
import numpy as np

In [36]:
# Data Extraction

# Read GeoJSON file using GeoPandas
data = gpd.read_file("/Users/ACER/Desktop/VIP_IDI/atl_bg.geojson")

# Extract polygons from the GeoJSON file
polygons = [i for i in data["geometry"]]

In [37]:
# Initialize lists to store entropy values and dictionaries
Entropys = []
dictionaries = []

# Loop through polygons and areas
for polygon in polygons:
    try:
        # Extract features from polygon using OSMnx
        gdf = ox.features.features_from_polygon(polygon, tags={'landuse': True})
        
        # Check if GeoDataFrame is empty
        if gdf.empty:
            Entropys.append(0)  # If empty, entropy is 0
            dictionaries.append({})  # Empty dictionary
            continue
    except:
        Entropys.append(0)  # If an exception occurs, entropy is 0
        dictionaries.append({})  # Empty dictionary
        continue
    
    # Convert GeoDataFrame to GeoDataFrame with geometry column
    gdf = gpd.GeoDataFrame(gdf, geometry='geometry')
    
    # Reset index
    gdf.reset_index(inplace=True)
    
    # Convert to CRS 4269
    gdf = gdf.to_crs(epsg=4269)
    
    # Calculate area for each geometry
    gdf["area"] = gdf.geometry.area
    
    # Calculate land use area dictionary
    landuse_area_dict = gdf.groupby('landuse')['area'].sum().to_dict()
    landuse_area_dict = {k: v for k, v in landuse_area_dict.items() if v != 0}
    dictionaries.append(landuse_area_dict)  # Append to dictionaries list
    
    # Remove certain land use types
    remove = ["aquaculture", "basin", "salt_pond", "railway"]
    valid_landuse_dict = {key: value for key, value in landuse_area_dict.items() if key not in remove}
    
    # Calculate total area
    area = sum(landuse_area_dict.values())
    
    # Calculate entropy
    k = len(valid_landuse_dict)
    if k > 1:
        denominator = np.log(k)
        numerator = 0
        for j in valid_landuse_dict.keys():
            p = valid_landuse_dict[j] / area
            numerator += p * np.log(p)
        E = -1 * numerator / denominator
        Entropys.append(E)
    else:
        Entropys.append(0)  # If only one land use type, entropy is 0


/var/folders/f7/jnv35vy91yn53q0rtcj3pklc0000gn/T/ipykernel_79754/3386545909.py:31: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["area"] = gdf.geometry.area
/var/folders/f7/jnv35vy91yn53q0rtcj3pklc0000gn/T/ipykernel_79754/3386545909.py:31: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["area"] = gdf.geometry.area
/var/folders/f7/jnv35vy91yn53q0rtcj3pklc0000gn/T/ipykernel_79754/3386545909.py:31: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["area"] = gdf.geometry.area
/var/folders/f7/jnv35vy91yn53q0rtcj3pklc0000gn/T/ipykernel_79754/3386545909.py:31: UserWarning: Geometry i

In [38]:
maxi = max(Entropys)

# Calculate the IDI 
LDI = [value / maxi for value in Entropys] 

In [39]:
with open('Atlanta_LDI_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Polygon', 'Land_use_dict','Entropy','LDI'])  # Header row

    # Use zip to combine lists into rows
    for polygon, dicts,E,v in zip(polygons, dictionaries, Entropys,LDI):
        writer.writerow([polygon, dicts,E,v])